## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [4]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [5]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [6]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [7]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [8]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [9]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [10]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [11]:
len(sentences_ted)

266694

In [11]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [20]:
# 'sentences_ted' is a list of list. If we directly use CountVectorizer,
# we will get an error 'AttributeError: 'list' object has no attribute 'lower''
# check out the answer here: https://stackoverflow.com/questions/52082477/python-list-error-for-count-vectorizer-and-fit-function
# I don't want to change the original code of the practical, so I just create a 
# new variable here.
sentences_ted_str = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted_str.append(' '.join(tokens))
print(len(sentences_ted_str))
print(sentences_ted_str[0])
print(sentences_ted_str[1])

266694
here are two reasons companies fail they only do more of the same or they only do what s new
to me the real real solution to quality growth is figuring out the balance between two activities exploration and exploitation


In [23]:
# count the words #
from sklearn.feature_extraction.text import CountVectorizer
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(sentences_ted_str)
# encode document
vector = vectorizer.transform(sentences_ted_str)
# summarize encoded vector
print(vector.shape)
print(type(vector))

(266694, 54418)
<class 'scipy.sparse.csr.csr_matrix'>


In [63]:
# get the top 1000 words #
# get the total count for each word
wc = vector.sum(axis=0)
print(wc.shape)
# get the top 1000. BTW, the type of 'wc' is numpy matrix 
wc.sort(axis=1)
counts_ted_top1000 = wc[0,-1000:]

(1, 54418)


Plot distribution of top-1000 words

In [61]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

/home/mengyah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  """Entry point for launching an IPython kernel.


One note here: 'CountVectorizer' ignores single characters during tokenization. Thus, the result above may be different from the result including the single characters. An example is shown here:

In [137]:
# here is Michael Leue's solution from https://github.com/mleue/oxford-deep-nlp-2017-solutions/blob/master/practical-1/practical1.ipynb
import collections
count = collections.Counter()
for sentence in sentences_ted:
    for word in sentence:
        count[word] += 1
words_top_ted = [token_count_pair[0] for token_count_pair in count.most_common(1000)]
counts_ted_top10001 = [token_count_pair[1] for token_count_pair in count.most_common(1000)]

In [66]:
print(counts_ted_top1000[0,-10:])
print(counts_ted_top10001[0:10])

[[ 63251  67629  70923  74738  78070  95146 114818 125169 149305 207748]]
[207748, 149305, 125169, 114818, 105399, 95146, 83180, 78070, 74738, 70923]


In [70]:
rev = {v: k for k, v in count.items()}
rev[83180]

'i'

As we can see, my result and Michael's result are different. This makes sense, as 'CountVectorizer' ignores single characters during tokenization. However, judging from the later parts, this practical does not want to ingore single character words like 'i'. Thus, I will just keep Michael's result here. 

### Part 3: Train Word2Vec

In [72]:
from gensim.models import Word2Vec

In [74]:
# for gensim, the input: Each sentence a list of words (utf8 strings):
model_ted = Word2Vec(sentences_ted, min_count=1) # this takes less than 1 minute on my machine

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [75]:
model_ted.most_similar("man")

/home/mengyah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.8406665325164795),
 ('guy', 0.816347062587738),
 ('lady', 0.7908732891082764),
 ('girl', 0.7598938345909119),
 ('soldier', 0.7566073536872864),
 ('gentleman', 0.7484695911407471),
 ('boy', 0.7469462156295776),
 ('david', 0.6906865835189819),
 ('poet', 0.6865618228912354),
 ('kid', 0.6824632883071899)]

In [76]:
model_ted.most_similar("computer")

/home/mengyah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('machine', 0.7571374177932739),
 ('software', 0.73600834608078),
 ('device', 0.6881433129310608),
 ('interface', 0.6734181046485901),
 ('robot', 0.6679286360740662),
 ('3d', 0.6551786661148071),
 ('printer', 0.6543788909912109),
 ('program', 0.6482037305831909),
 ('camera', 0.6397494077682495),
 ('game', 0.6228448152542114)]

In [77]:
model_ted.most_similar("love")

/home/mengyah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('hate', 0.6153411865234375),
 ('fear', 0.5587862133979797),
 ('feel', 0.5500969886779785),
 ('poetry', 0.5359469056129456),
 ('sing', 0.5343461632728577),
 ('hope', 0.5247131586074829),
 ('hear', 0.5203284025192261),
 ('lunchmate', 0.5190479755401611),
 ('joy', 0.5129758715629578),
 ('loved', 0.5081714391708374)]

In [78]:
model_ted.most_similar("i")

/home/mengyah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('myself', 0.6268762350082397),
 ('you', 0.5369234085083008),
 ('we', 0.5291538238525391),
 ('dad', 0.49471062421798706),
 ('godammit', 0.4769575595855713),
 ('voice', 0.4634573757648468),
 ('mom', 0.45205172896385193),
 ('newsfeed', 0.4488353133201599),
 ('poetry', 0.44860103726387024),
 ('she', 0.4369257986545563)]

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [79]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted] # 'words_top_ted' has been calculated in part 2

/home/mengyah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [80]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [81]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [82]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [83]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [84]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [85]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [139]:
# sentences_wiki is a list of strings. Word2Vec needs the input as list of list, so we nned to tokenize them
sentences_wiki_t = []
for sent_str in sentences_wiki:
    tokens =  re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_wiki_t.append(tokens)
# count the words
count_wiki = collections.Counter()
for sentence in sentences_wiki_t:
    for word in sentence:
        count_wiki[word] += 1 
word_top_wiki = []
count_top_wiki = []
for token_count_pair in count_wiki.most_common(1000):
    words_top_wiki.append(token_count_pair[0])
    count_top_wiki.append(token_count_pair[1])
# plot the top 1000
hist, edges = np.histogram(count_top_wiki, density = True, bins = 100, normed = True)
p = figure(tools = 'pan, wheel_zoom, reset, save',
          toolbar_location='above',
          title='Top 1000 words distribution for WIKI data')
p.quad(top= hist, bottom=0, left=edges[:-1], right=edges[1:], line_color='#555555')
show(p)

/home/mengyah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.


In [142]:
word_top_wiki[0:10]

['the', 'of', 'and', 'in', 'to', 'a', 'was', 's', 'on', 'as']

In [143]:
# train Word2Vec
model_wiki = Word2Vec(sentences_wiki_t,min_count=1,workers=4) # workers means running the training parallelly

Note: remember to tokenize the sentences before train Word2Vec. If you directly uses sentences_wiki,which a list of strings, your resulting vocabulary of model_wiki would be just letters rather than words. (Michael's solution has this problem, so he gets the error of out of vocabulary.)

In [144]:
model_wiki.most_similar('man')

/home/mengyah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.7038487195968628),
 ('girl', 0.6593668460845947),
 ('creature', 0.6336866021156311),
 ('person', 0.6142929792404175),
 ('boy', 0.6126394271850586),
 ('figure', 0.5927751064300537),
 ('dog', 0.5917795896530151),
 ('kid', 0.5871160626411438),
 ('someone', 0.5635063648223877),
 ('soldier', 0.5631564259529114)]

In [145]:
model_wiki.most_similar('computer')

/home/mengyah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('software', 0.774712085723877),
 ('user', 0.7229084968566895),
 ('hardware', 0.7180997133255005),
 ('computers', 0.7060397863388062),
 ('zx', 0.6991461515426636),
 ('simulation', 0.6955831050872803),
 ('puzzle', 0.6907745599746704),
 ('interactive', 0.6841230988502502),
 ('technology', 0.681239902973175),
 ('handheld', 0.678767204284668)]

In [146]:
model_wiki.most_similar('i')

/home/mengyah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('we', 0.7434115409851074),
 ('you', 0.7293256521224976),
 ('somebody', 0.6267027854919434),
 ('me', 0.6024430990219116),
 ('kgnc', 0.5800503492355347),
 ('genevi', 0.5582752823829651),
 ('yourself', 0.5485366582870483),
 ('tacandra', 0.5472677946090698),
 ('nobody', 0.5419475436210632),
 ('toussieng', 0.5404810905456543)]

#### t-SNE visualization

In [147]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

/home/mengyah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [148]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)